# Library

In [1]:
#import sys
import pandas as pd
import numpy as np
import random
import pyspark
import itertools
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
import pickle
import statsmodels.api as sm

from pyspark import SparkContext, SQLContext

from math import sqrt
from time import time as ttt

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier as DTC_spark
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import joblib
from joblib import parallel_backend
from joblib import Parallel, delayed
from joblib import parallel_backend


# Upload

In [2]:
spark = SparkSession.builder.master("Yarn").appName("spark_app_1234").getOrCreate()
sc = spark.sparkContext
spark
d0 = (
    spark
    .read
    .format("csv") 
    .option("header","true") 
    .option("inferSchema","true") 
    .load("gs://mas-a5-storage-1/notebooks/jupyter/application_train.csv")
)

d1 = d0.filter(d0.DAYS_EMPLOYED != 365243).select('TARGET','DAYS_EMPLOYED')

print(d1.rdd.getNumPartitions())

4


In [3]:
sc

<SparkContext master=yarn appName=PySparkShell>

# Function

In [4]:
def prepare_spark_data(n_part, k_mult):
    '''
    takes 'DAYS_EMPLOYED and 'TARGET'  from d1 (alreay filtered)
    rearrane in n partitions (if n==0 keeps initial number of partitions)
    prints final shape/ partition
    returns d2 - spark df
    '''
    data = d1
    data_new = data # first step in the cycle
    for i in range(k_mult-1):
        data_tmp = data.select('TARGET', \
                     f.col('DAYS_EMPLOYED')*(f.lit(0.9995) + f.rand()/1000)).\
                   toDF('TARGET','DAYS_EMPLOYED')
        data_tmp = data_tmp.select('TARGET', f.floor('DAYS_EMPLOYED'))
        data_new = data_new.union(data_tmp)

    assembler = VectorAssembler(inputCols=["DAYS_EMPLOYED"], 
                        outputCol="DAYS_EMPLOYED_vect")
    d2 = assembler.transform(data_new)
    if n_part != 0:
        d2 = d2.repartition(n_part)       
    #print(f'n-partitions initial: {d2.rdd.getNumPartitions()}; df size: {d2.count()}\n')

    return d2

# Params an run

In [ ]:
## !! mind the fn NOT TO rewrite results
fn = '4cpu_by_4n__4m_yarn_ssd'
size_mult = [1, 2, 5, 10, 15, 20, 30, 50, 100]
partitions = [2, 4, 8, 16] 
# initial, 2**i incl n_nodes till n_cpu*n_nodes and n_cpu*n_nodes
n_iter = 3

print('d1-size', d1.count())
n_part_base = d1.rdd.getNumPartitions()
print('n-partitions initial', n_part_base, '\n')

rd1 = {}
for k_size_mult in size_mult:
    rd2 = {}
    for n_part in partitions:
        df = prepare_spark_data(n_part, k_size_mult)
        df.cache()
        print('======================================================================')
        print(f'=== size_mult={k_size_mult}; \
        n_part_req={n_part}; n_part_act={df.rdd.getNumPartitions()}; df_count={df.count()} ===\n')
        times = [0 for i in range(n_iter)]
        models = {}
        for i in range(n_iter):
            dt = DTC_spark(labelCol="TARGET",
                       featuresCol="DAYS_EMPLOYED_vect",
                       minInfoGain=0.0001,
                       impurity='entropy',
                       maxDepth=14, maxBins=2**14, # it differs from scikit learn - it means number of canidate split points
                       #minInstancesPerNode = 1,
                       #checkpointInterval = 10
                       )
           
            t0 = ttt()
            model = dt.fit(df)
            t1 = ttt()
            times[i] = t1-t0
            models[i] = model 
            print('model', i, 'build time', round(times[i],2), '\n', model)
        rd2[n_part] = (models, times)
    rd1[k_size_mult] = rd2

result = rd1  

d1-size 252137
n-partitions initial 4 



=== size_mult=1;         n_part_req=2; n_part_act=2; df_count=252137 ===



model 0 build time 6.93 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_b30adb956b80, depth=14, numNodes=123, numClasses=2, numFeatures=1


model 1 build time 6.48 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_f2d4a7ccf821, depth=14, numNodes=123, numClasses=2, numFeatures=1


model 2 build time 6.46 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_743212ae4e3c, depth=14, numNodes=123, numClasses=2, numFeatures=1


...

model 2 build time 32.25 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_784a5e1b8af4, depth=12, numNodes=47, numClasses=2, numFeatures=1


=== size_mult=100;         n_part_req=16; n_part_act=16; df_count=25213700 ===



model 0 build time 37.88 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_5ee2c18a9caa, depth=10, numNodes=35, numClasses=2, numFeatures=1


model 1 build time 29.57 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_1b84bb484f88, depth=10, numNodes=35, numClasses=2, numFeatures=1


model 2 build time 26.45 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_07638440d27d, depth=10, numNodes=35, numClasses=2, numFeatures=1


# Results

In [10]:
di = result
df_res_time = pd.DataFrame()
df_res_nodes = pd.DataFrame()
for p1 in di.keys():
    for p2 in di[p1].keys():
        df_res_time.loc[p1, p2] = np.round(np.median(di[p1][p2][1]), 1)
        m_tmp = di[p1][p2][0]
        n_nodes = []
        for i,_ in enumerate(m_tmp):
            tmp = f'{m_tmp[i]}'.split(' ')
            print(i, tmp)
            n_nodes.append([int(x.split('=')[1][:-1]) for x in tmp if x[:4]== 'numN'][0])
        df_res_nodes.loc[p1, p2] = np.round(np.mean(n_nodes), 1)

df_res_time.to_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_t.csv')
df_res_nodes.to_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_n.csv')

display(df_res_time)      
display(df_res_nodes)     

0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_b30adb956b80,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
1 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_f2d4a7ccf821,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
2 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_743212ae4e3c,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_c2b28cc4ad08,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
1 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_1d3589241bea,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
2 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_7b4a57dc6944,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_fafd2a48ffa6,', 'depth=14,', 'numNodes=123,', 'numC

,2,4,8,16
1,6.5,7.7,10.1,15.4
2,8.6,5.6,6.6,10.3
5,14.8,11.0,8.8,22.1
10,20.5,12.2,9.1,10.3
15,25.9,15.2,9.3,10.8
20,28.6,16.7,11.2,10.7
30,39.5,22.1,15.3,13.9
50,57.6,30.2,19.0,17.5
100,90.4,61.2,34.9,29.6


,2,4,8,16
1,123.0,123.0,123.0,123.0
2,125.0,121.0,89.0,83.0
5,85.0,55.0,73.0,83.0
10,69.0,51.0,107.0,99.0
15,51.0,51.0,69.0,35.0
20,39.0,59.0,41.0,49.0
30,63.0,45.0,37.0,49.0
50,39.0,41.0,37.0,59.0
100,39.0,33.0,47.0,35.0


In [11]:
fn = '4cpu_by_8n__4m_yarn_ssd'

tmp = pd.read_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_t.csv')
tmp

,Unnamed: 0,2,4,8,16
0,1,6.5,7.7,10.1,15.4
1,2,8.6,5.6,6.6,10.3
2,5,14.8,11.0,8.8,22.1
3,10,20.5,12.2,9.1,10.3
4,15,25.9,15.2,9.3,10.8
5,20,28.6,16.7,11.2,10.7
6,30,39.5,22.1,15.3,13.9
7,50,57.6,30.2,19.0,17.5
8,100,90.4,61.2,34.9,29.6


In [12]:
fn = '4cpu_by_8n__4m_yarn_ssd'
tmp = pd.read_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_n.csv')
tmp

,Unnamed: 0,2,4,8,16
0,1,123.0,123.0,123.0,123.0
1,2,125.0,121.0,89.0,83.0
2,5,85.0,55.0,73.0,83.0
3,10,69.0,51.0,107.0,99.0
4,15,51.0,51.0,69.0,35.0
5,20,39.0,59.0,41.0,49.0
6,30,63.0,45.0,37.0,49.0
7,50,39.0,41.0,37.0,59.0
8,100,39.0,33.0,47.0,35.0
